В нашем распоряжении имеется размеченный датасет с набором комментариев к товарам интернет-магазина.  
Задача данной работы - обучить несколько моделей классификации отзывов на позитивные и негативные.


Столбец *text* - текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import pandas as pd
import numpy as np
import sys
import nltk # Natural Language Toolkit
from nltk.stem import PorterStemmer  # стеммер для английского языка
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import re
from tqdm import notebook # для отображения времени работы процесса
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import f1_score

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Выделим корпус текстов

In [5]:
corpus = data[['text']]

In [6]:
porter_stemmer=nltk.PorterStemmer()

In [7]:
# функция для очистки текста от лишних знаков без стеммера

def clear_text(text):
    pattern = r'[^a-zA-z0-9]' # шаблон для лишних знаков
    text = re.sub(pattern, ' ', text) # заменяем пробелами неподходящий под шаблон текст
    text = text.lower() # перевод слов в нижний регистр
    text_list = text.split()
    text = [porter_stemmer.stem(text) for text in text_list]
    text_str = ' '.join(text)
    return text_str

In [8]:
sys.setrecursionlimit(7500)
# иначе возникает ошибка RecursionError: maximum recursion depth exceeded in comparison

Очистим данные от лишних знаков

In [9]:
#for i in notebook.tqdm(range(len(corpus_test))):
corpus_stemmed = corpus['text'].apply(clear_text)

In [10]:
corpus_stemmed.shape

(159571,)

Добавим очищенный и обработанный стеммером набор текстов в исходный датасет

In [11]:
data['text_stemmed'] = corpus_stemmed

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 3 columns):
text            159571 non-null object
toxic           159571 non-null int64
text_stemmed    159571 non-null object
dtypes: int64(1), object(2)
memory usage: 3.7+ MB


Разделим выборку на обучающую и тестовую

In [13]:
feature = data['text_stemmed']
target = data['toxic']

In [14]:
feature_train, feature_test, target_train, target_test = train_test_split(feature, target, test_size=0.33, random_state=128)

In [15]:
feature_train.shape

(106912,)

In [16]:
feature_test.shape

(52659,)

Создадим стоп-слова

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
stop_words = set(stopwords.words("english"))

Создадим матрицу TF-IDF по корпусу текстов

In [19]:
tf_idf_vector = TfidfVectorizer(stop_words=stop_words)

In [20]:
tf_idf = tf_idf_vector.fit_transform(feature_train)

# 2. Обучение

#### Обучим модель логистической регрессии

In [21]:
model = LogisticRegression(random_state=128, class_weight='balanced')
model.fit(tf_idf, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=128, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

Посчитаем TF-IDF для тестовой выборки. Применим только метод transform.

In [22]:
tf_idf_test = tf_idf_vector.transform(feature_test)

In [23]:
predictions = model.predict(tf_idf_test)

In [24]:
type(predictions)

numpy.ndarray

In [25]:
type(predictions)

numpy.ndarray

In [26]:
f1 = f1_score(target_test, predictions)

In [27]:
print(f1)

0.7507855134777576


#### Обучим модель Complement Naive Bayes

Создадим мешок слов и исключим из расчетов стоп-слова

In [28]:
count_vect = CountVectorizer(stop_words=stop_words)
bow = count_vect.fit_transform(feature_train)

In [29]:
clf = ComplementNB()

In [30]:
clf.fit(bow, target_train)

ComplementNB(alpha=1.0, class_prior=None, fit_prior=True, norm=False)

In [31]:
bow_test = count_vect.transform(feature_test)

In [32]:
predictions = clf.predict(bow_test)

In [33]:
f1 = f1_score(target_test, predictions)

In [34]:
print(f1)

0.681523074223548


# 3. Выводы

В данном проекте было проведено исследование двух моделей для классификации тональности текстов:
 - модель логистической регресси с использованием метода TF-IDF показала значение метрики F1=0.751
 - модель Complement Naive Bayes c использованием только CountVectorizer показала F1 = 0.681.

